## 1. Tensor

 - 텐서플로우를 구성하는 가장 기초적인 데이터 단위
 - n 차원 배열의 집합 또는 n 차원 배열을 의미
 - rank: 텐서의 차원
 - 텐서의 표현은 `numpy` 배열을 사용

`3. # a rank 0 tensor; a scalar with shape []
[1., 2., 3.] # a rank 1 tensor; a vector with shape [3],
[[1., 2., 3.], [4., 5., 6.]] # a rank 2 tensor; a matrix with shape [2, 3],
[[[1., 2., 3.]], [[7., 8., 9.]]] # a rank 3 tensor with shape [2, 1, 3]`

## 2. TensorFlow programming

 #### 1) 연산 그래프(computational graph) 정의
  - 연산그래프는 텐서플로우 작업을 순차적으로 정의(표현)한 것으로 노드와 에지를 갖는 그래프 형태를 갖음 
  - 연산그래프의 노드에는 텐서를 입력값으로 받아 연산하는 작업들이 위치 : `tf.Operation`
  - 연산그래프의 에지에는 노드에 정의된 연산간에 주고 받는 데이터 들을 표현(텐서들이 그래프 상에서 흐름.) `tf.Tensor`

 #### 2) 연산 그래프를 실행
  - 연산그래프의 실행은 `tf.Session` 객체,텐서플로우가 실행되는 환경을 만들어서 진행됨
  - 연산그래프의 작업을 CPU, GPU에 배정하고 실행을 위한 메서드를 제공

#### [default 그래프에 정의하기]
 - 3개의 노드(2개: constant op, 1개 matmul op)
 - 특정 그래프 객체에 명시적으로 연산을 정의하지 않는한 모든 연산은 전역 default 그래프에 정의됨 

In [ ]:
import tensorflow as tf
mat_a = tf.constant([[3.0, 3.0]], dtype=tf.float32)
mat_b = tf.constant([[2.0],[2.0]], dtype=tf.float32)
product = tf.matmul(mat_a, mat_b)

print(tf.get_default_graph() is product.graph)

#### [특정 그래프에 연산 정의하기]

In [ ]:
g_1 = tf.Graph()
with g_1.as_default():
    mat_a = tf.constant([[3.0, 3.0]], dtype=tf.float32)
    mat_b = tf.constant([[2.0],[2.0]], dtype=tf.float32)
    product = tf.matmul(mat_a, mat_b)
    print(product.graph is g_1)

g_2 = tf.Graph()
with g_2.as_default():
    mat_a = tf.constant([[3.0, 3.0]], dtype=tf.float32)
    mat_b = tf.constant([[2.0],[2.0]], dtype=tf.float32)
    product = tf.matmul(mat_a, mat_b)
    print(product.graph is g_2)
# with tf.Graph().as_default() as g_2:
#     mat_a = tf.constant([[3.0, 3.0]], dtype=tf.float32)
#     mat_b = tf.constant([[2.0],[2.0]], dtype=tf.float32)
#     product = tf.matmul(mat_a, mat_b)
#     print(product.graph is g_2)

#### [그래프 실행하기]
 - session 객체의 run 매서드 호출
 - default 그래프에 정의한 3개의 작업이 실행 (graph=None)
 - 사용한 session 반환

In [ ]:
sess = tf.Session(graph=g_2)
print(sess.run(product))
sess.close()

 - session 컨텍스트 매니저 활용

In [ ]:
with tf.Session(graph=g_2) as sess:
    print(sess.run(product))

 - 연산 자원 할당

In [ ]:
with tf.Session(graph=g_2) as sess:
    with tf.device('/gpu:0'):
        print(sess.run(product))

#### [인터렉티브한 이용]
  - Tensor.eval(), Operation.run() 메서드 활용

In [ ]:
import tensorflow as tf
sess = tf.InteractiveSession()
x = tf.Variable([1.0,2.0])
a = tf.constant([3.0,3.0])
x.initializer.run()
sub = tf.subtract(x,a)
print(sub.eval())
sess.close()

## 3. TensorFlow tf.constant, tf.Variable

 - 연산 그래프에 정의된 연산을 수행하기 위해 필요한 데이터 값을 입력 위한 수단 

 ### 3-1. tf.constant
 - 상수 텐서를 생성하는 작업으로, `tf.constant` 연산 정의시 제공한 초기값을 갖는 텐서를 반환

 ### 3-2. tf.Variable
 - 텐서플로우 프로그램에서 연산의 결과가 공유되고, 상태를 유지해야하는 경우 사용 
   - ex) 학습을 진행하면서 모델의 파라미터가 업데이트 되야하므로 모델의 파라미터를 변수로 표현
 - 변수 연산을 정의하기 위해 텐서를 초기값으로 부여, 초기값으로 제공한 텐서로 변수 type과 shape이 결정됨
 - 변수 연산이 정의되면 타입과 변수 type과 shape은 고정됨, 변수 값인 텐서를 assign 메서드로 변경
 - 연산을 실행하기 전, 그래프 상에 정의된 변수를 명시적으로 초기화하는 작업 필요
   - 초기화 연산을 실행(`tf.global_variable_initializer()`), 변수 값이 저장된 파일에서 복구, `assign` 메서드 실행

In [ ]:
state = tf.Variable(0, name="counter")
one = tf.constant(1)
new_value = tf.add(state, one)
update = tf.assign(state, new_value)
init_op = tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init_op)
    print(sess.run(state))
    for _ in range(3):
        sess.run(update)
        print(sess.run(state))

 #### [변수의 저장과 복구]
  - 변수를 이름과 텐서 값을 매핑해놓은 바이너리 파일(`.ckpt`)에 저장 가능
  - `tf.train.Saver()` 객체를 이용하여 그래프 전체 변수와 지정된 리스트 변수를 저장하고 복구
  - 저장될 때 사용되는 변수 명은 `Variable.name`이 기본 값
  - `tf.train.Saver()` 객체에 딕셔너리를 저장할 이름(key), 저장할 값(value)로 전달하여 저장시 사용할 이름을 변경하거나 변수를 선택적으로 저장 가능
    - ex) `tf.train.Saver({"saved_v":v})`
  - 전체 변수를 파일에서 복구 시 변수 초기화가 필요 없음

In [ ]:
# Create some variables.
import tensorflow as tf
import os, shutil

with tf.variable_scope("variable", reuse=tf.AUTO_REUSE):
    v1 = tf.get_variable("v1", shape=[3], initializer = tf.zeros_initializer)
    v2 = tf.get_variable("v2", shape=[5], initializer = tf.zeros_initializer)

inc_v1 = v1.assign(v1+1)
dec_v2 = v2.assign(v2-1)

# Add an op to initialize the variables.
init_op = tf.global_variables_initializer()

# Add ops to save and restore all the variables.
saver = tf.train.Saver()

# Later, launch the model, initialize the variables, do some work, and save the
# variables to disk.
with tf.Session() as sess:
  sess.run(init_op)
  # Do some work with the model.
  inc_v1.op.run()
  dec_v2.op.run()
  # Save the variables to disk.
  shutil.rmtree("./tmp/ckpt")
  os.mkdir("./tmp/ckpt")
  save_path = saver.save(sess, "./tmp/ckpt/model.ckpt")
  print("Model saved in path: %s" % save_path)
    
    
from tensorflow.python.tools import inspect_checkpoint as chkp
chkp.print_tensors_in_checkpoint_file(save_path, tensor_name='',  all_tensors=True)

In [ ]:
tf.reset_default_graph()

# Create some variables.
with tf.variable_scope("variable", reuse=tf.AUTO_REUSE):
    v1 = tf.get_variable("v1", shape=[3])
    v2 = tf.get_variable("v2", shape=[5])
    
# Add ops to save and restore all the variables.
saver = tf.train.Saver()

# Later, launch the model, use the saver to restore variables from disk, and
# do some work with the model.
with tf.Session() as sess:
  # Restore variables from disk.
  saver.restore(sess, "./tmp/ckpt/model.ckpt")
  print("Model restored.")
  # Check the values of the variables
  print("v1 : %s" % v1.eval())
  print("v2 : %s" % v2.eval())

 ### - tf.get_variable()
  - 공유 변수를 손쉽게 구현하기 위해 `tf.Variable`를 직접호출하지 않고 생성된 변수를 가져오거나 존재하지 않을 시 새롭게 생성
     - ex) 매우 깊은 층을 갖는 심층심경망 네트워크 구현시 각 층마다 변수를 정의하는 데 따른 불편함을 해결

In [ ]:
with tf.variable_scope("layer1", reuse=tf.AUTO_REUSE):
     weight = tf.get_variable(
            "weight", 
            [1, 2, 3], 
            dtype=tf.int32, 
            initializer=tf.zeros_initializer
    )

## 4. Fetches

 - 그래프 상에 정의된 작업 하나 이상의 작업 결과 가져오기

In [ ]:
input1 = tf.constant([3.0])
input2 = tf.constant([2.0])
input3 = tf.constant([5.0])
intermed = tf.add(input2, input3)
mul = tf.multiply(input1, intermed)

with tf.Session() as sess:
  result = sess.run([mul, intermed])
  print(result)

## 5. Feed

 - 실행 시점에 연산 그래프 상으로 텐서 값을 제공하는 매커니즘
 - tf.placeholder를 이용 텐서가 입력될 공간을 확보

In [ ]:
import numpy as np
x1 = tf.placeholder(tf.float32, shape=(4, 4), name='input1')
x2 = tf.placeholder(tf.float32, shape=(4, 4), name='input2')
y = tf.matmul(x1, x2)

with tf.Session() as sess:
#   print(sess.run(y))  # ERROR: will fail because x was not fed.
  arr = np.random.rand(4, 4)
  print(sess.run(y, feed_dict={x1: arr, x2:arr}))

## 6. Visualization : Tensorboard

 - 텐서플로우가 제공하는 텐서보드를 활용하여 연산 그래프를 시각화
 - 그래프 실행 후 연산 결과 시각화
 - `tf.summary.FileWriter()` 객체에 연산 그래프와 연산 결과 값을 저장 후 텐서보드로 시각화   
 - 벡터: `tf.summary.histogram()`, 스칼라: `tf.summary.scalar()`로 시각화할 연산 값 설정
 - 텐서보드 실행 => `tensorboard --logdir="./logs/xor_log"` 

In [ ]:
import tensorflow as tf
import numpy as np

learning_rate = 0.01

x_data = [[0, 0],
          [0, 1],
          [1, 0],
          [1, 1]]
y_data = [[0],
          [1],
          [1],
          [0]]
x_data = np.array(x_data, dtype=np.float32)
y_data = np.array(y_data, dtype=np.float32)

X = tf.placeholder(tf.float32, [None, 2], name='x-input')
Y = tf.placeholder(tf.float32, [None, 1], name='y-input')

with tf.name_scope("layer1"):
    W1 = tf.Variable(tf.random_normal([2, 2]), name='weight1')
    b1 = tf.Variable(tf.random_normal([2]), name='bias1')
    layer1 = tf.sigmoid(tf.matmul(X, W1) + b1)

    w1_hist = tf.summary.histogram("weights1", W1)
    b1_hist = tf.summary.histogram("biases1", b1)
    layer1_hist = tf.summary.histogram("layer1", layer1)


with tf.name_scope("layer2"):
    W2 = tf.Variable(tf.random_normal([2, 1]), name='weight2')
    b2 = tf.Variable(tf.random_normal([1]), name='bias2')
    hypothesis = tf.sigmoid(tf.matmul(layer1, W2) + b2)

    w2_hist = tf.summary.histogram("weights2", W2)
    b2_hist = tf.summary.histogram("biases2", b2)
    hypothesis_hist = tf.summary.histogram("hypothesis", hypothesis)

# cost/loss function
with tf.name_scope("cost"):
    cost = -tf.reduce_mean(Y * tf.log(hypothesis) + (1 - Y) *
                           tf.log(1 - hypothesis))
    cost_summ = tf.summary.scalar("cost", cost)

with tf.name_scope("train"):
    train = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

# Accuracy computation
# True if hypothesis>0.5 else False
predicted = tf.cast(hypothesis > 0.5, dtype=tf.float32)
accuracy = tf.reduce_mean(tf.cast(tf.equal(predicted, Y), dtype=tf.float32))
accuracy_summ = tf.summary.scalar("accuracy", accuracy)

# Launch graph
with tf.Session() as sess:
    # tensorboard --logdir=./logs/xor_logs
    merged_summary = tf.summary.merge_all()
    writer = tf.summary.FileWriter("./logs/xor_log")
    writer.add_graph(sess.graph)  # Show the graph

    # Initialize TensorFlow variables
    sess.run(tf.global_variables_initializer())

    for step in range(10001):
        summary, _ = sess.run([merged_summary, train], feed_dict={X: x_data, Y: y_data})
        writer.add_summary(summary, global_step=step)

        if step % 100 == 0:
            print(step, sess.run(cost, feed_dict={
                  X: x_data, Y: y_data}), sess.run([W1, W2]))

    # Accuracy report
    h, c, a = sess.run([hypothesis, predicted, accuracy],
                       feed_dict={X: x_data, Y: y_data})
    print("\nHypothesis: ", h, "\nCorrect: ", c, "\nAccuracy: ", a)


In [ ]:
#!tensorboard --logdir='./logs/xor_log'